In [2]:
import csv
import re
import string

In [3]:
from timeit import default_timer as timer

In [ ]:
def preprocess_text(text):
    """
    """
    text = text.strip().lower()
    text = re.sub('\d+|\d+\.\d+', '[NUM]', text)

    # PHI tags are in format [**INFORMATION**]
    text = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', text)
#     phi_tags = re.findall('(\[\*\*.*?\*\*\])', text)
#     for i, tag in enumerate(phi_tags):
#         text = text.replace(tag, '__PHI_{}__'.format(i))
        
    text = re.sub('---+', '\n\n-----\n\n', text)
    text = re.sub('___+', '\n\n_____\n\n', text)
    text = re.sub('[\?\.\!]+(?=[\?\.\!])', '', text)
    text = re.sub('\\+', ' ', text)
    text = re.sub('-|/', '_', text)
    text = re.sub('_+', '_', text)
    text = re.sub('\*|\(|\)', ' ', text)
    text = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', text)
    text = re.sub('\s_\s', ' ', text)
    text = re.sub('\r+|\n+|\u0085+|\u2028+|\u2029+', ' ', text) # Replace newlines
    text = re.sub('\s+|\u00A0+', ' ', text) # Replace multiple spaces w/ single    
    segments = text.split(' ')
    
    return segments

In [ ]:
NOTEEVENTS = "../data/NOTEEVENTS.csv"

In [ ]:
notes = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        note = preprocess_text(note[-1])
        notes.append(note)
#         if note[6] == 'Discharge summary':
#             note = preprocess_text(note[-1])
#             notes.append(note)

In [ ]:
import gensim
from gensim.models.fasttext import FastText
from timeit import default_timer as timer

In [ ]:
model = FastText(size=300, window=10, min_count=5)

In [ ]:
model.build_vocab(lines)

In [ ]:
print(model)

In [ ]:
start = timer()

model.train(lines, total_examples=len(lines), epochs=5)

end = timer()
print(end - start)

In [ ]:
print(model)

In [ ]:
model.save('saved_model_gensim2')

In [ ]:
model.wv.doesnt_match(['cipro', 'dexamethasone', 'radiology', 'sitagliptin'])

In [ ]:
with open("processed_text.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(notes)

In [ ]:
def clean_string1(text):
    """
    """
    text = text.strip().lower().replace('-', '_').replace('.', '_').replace(' ', '_').rstrip('_')
    return text

def preprocess_text2(query):
    """
    """
    query = re.sub('\d+|\d+\.\d+', '[NUM]', query)
    query = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', query)
    query = query.strip('"').strip('?').strip("'").strip('(').strip(')').strip(':')
    query = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', query)
    word_list = query.split()
    word_list = [clean_string1(word) for word in word_list]
    return word_list

In [ ]:
notes2 = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        note = preprocess_text2(note[-1])
        notes2.append(note)

In [ ]:
with open("processed_text.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(notes2)

In [4]:
notes2 = []
with open("processed_text.csv", "r") as f:
    reader = csv.reader(f)
    for note in reader:
        notes2.append(note)

In [ ]:
notes2[2]

## FastText model

In [6]:
import gensim
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
from timeit import default_timer as timer

In [7]:
ft_model = FastText(size=300, window=10, min_count=5)

In [8]:
start = timer()

ft_model.build_vocab(notes2)

end = timer()
print(end-start)
print(ft_model)

638.9453610269993
FastText(vocab=208155, size=300, alpha=0.025)


In [9]:
start = timer()

ft_model.train(notes2, total_examples=len(notes2), epochs=5)

end = timer()
print(end - start)

25857.242537287995


In [10]:
ft_model.save('fasttext_embeddings')

## Word2Vec model

In [12]:
w2v_model = Word2Vec(size=300, window=10, min_count=5, sg=1)

In [13]:
start = timer()

w2v_model.build_vocab(notes2)

end = timer()
print(end-start)
print(ft_model)

355.62777747400105
FastText(vocab=208155, size=300, alpha=0.025)


In [14]:
start = timer()

w2v_model.train(notes2, total_examples=len(notes2), epochs=5)

end = timer()
print(end - start)

15519.88943515101


In [15]:
w2v_model.save('w2v_embeddings')